<a href="https://colab.research.google.com/github/himanshu-nakrani/TMLC-Gen-ai-projects/blob/main/Langchain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain cohere langchain_community google-search-results

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=ea3e1541a389f020108bd61f339bc46d504d1ff8b3ebd099027286f6168de863
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstal

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_community.llms import Cohere
from langchain import LLMMathChain
from google.colab import userdata

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('serp')

In [ ]:
# model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

react_agent_llm = Cohere(cohere_api_key=userdata.get('COHERE_KEY'))
math_chain_llm = Cohere(cohere_api_key=userdata.get('COHERE_KEY'))

tools = load_tools(["serpapi"], llm=react_agent_llm)

llm_math_chain = LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

<ipython-input-4-f26aa838e9e5>:3: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  react_agent_llm = Cohere(cohere_api_key=userdata.get('COHERE_KEY'))


In [ ]:
react_agent = initialize_agent(tools,
                               react_agent_llm,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True,
                               )

<ipython-input-5-fda2adc2dab0>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  react_agent = initialize_agent(tools,


In [ ]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["Search", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

In [ ]:
react_agent("When did the last Fifa world cup happen and who won it?")



> Entering new AgentExecutor chain...
 Thought: Assuming you would like to know about the winner of the most recent World Cup, as that is the most common interpretation of "last". I should look up the most recent FIFA World Cup finals. 

Action: Search
Action Input: Last FIFA World Cup winners

Observation: {'title': '2022 World Cup', 'games': [{'tournament': 'World Cup', 'stage': 'Final', 'venue': 'Lusail Stadium', 'venue_kgmid': '/m/0fq1625', 'status': 'FT (P)', 'date': 'Dec 18, 22', 'video_highlights': {'link': 'https://www.plus.fifa.com/content/46772b92-24c3-4315-87da-158ef7af6259?cid=(p_go-box)(c_FIFAPlus)(sc_highlights)(ssc_)(da_)(co_)(cc_)&s_cid=(co_)(cc_)&c=(c_FIFAPlus)(sc_highlights)(ssc_)(cc_)&pid=(p_go-box)(ch_)(t_)', 'thumbnail': 'https://serpapi.com/searches/67697eb92bef57696834e091/images/0471cf2577eb7795d91dfc8dd2e8c7fdd176b0bcf269b4b7dca7ba96bbe20e3a6b14bf2d412367087ba4054e1f714786e2c2e435cdc5692d.jpeg', 'duration': '2:00'}, 'teams': [{'name': 'Argentina', 'score': '3

{'input': 'When did the last Fifa world cup happen and who won it?',
 'output': 'Argentina'}